In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu

# Constants
G = cc.G.si.value  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun.si.value  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = G*M  # gravitational parameter for the Sun in m^3/s^2

# Constants for Mercury
a_mercury = 57.909e9 * uu.m  # Semi-major axis of Mercury's orbit in meters
e = e_mercury = 0.205630  # Eccentricity of Mercury's orbit
# Specific angular momentum (h) for Mercury
# h = sqrt(G * M * a * (1 - e^2))
h_mercury = ((cc.G * cc.M_sun * a_mercury * (1 - e_mercury**2))**0.5).decompose()
h = h_mercury.to(uu.m**2 / uu.s).value  # Convert to m^2/s for consistency with previous units used




In [8]:
def r_derivatives(theta, h, e, GM, x):
    sigma = x[0]
    k = x[1]
    # Expression for r, r_dot, and r_double_dot
    r = (h ** 2 / GM) / (1 + k * e * np.cos(theta / sigma))
    r_dot = -GM*e*np.sin(theta/sigma)/(h*sigma)
    r_double_dot =  GM**3*e*(e*np.cos(theta/sigma) + 1)**2*np.cos(theta/sigma)/(h**4*sigma**2)
    theta_dot = h / r**2
    return r, r_dot, r_double_dot, theta_dot


def error_function(x):
    def integrand(theta):
        r, r_dot, r_double_dot, theta_dot = r_derivatives(theta, h, e, GM, x)
        v_squared = r_dot**2 + r**2 * theta_dot**2
        v = np.sqrt(v_squared)
        c = 299792458
        gamma_v = 1/np.sqrt(1-v**2/c**2)
        a_theoretical = -GM/gamma_v / r**2 /(1+(gamma_v-1)*(r_dot / v)**2)
#         a_theoretical = -GM/ r**2
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2
    integral_error, _ = quad(integrand, 0, 2*np.pi, epsabs=1e-14, epsrel=1e-14)
    return 100*integral_error

# Initial guess for sigma
initial_x = [0.9, 1.0]
result = minimize(error_function, initial_x, method='Nelder-Mead', tol=1E-14)
print("Optimized sigma:", result.x)


# Constants
orbital_period_days = 88  # Orbital period of Mercury in days
days_per_year = 365.25  # Average number of days per year, accounting for leap years
years_per_century = 100  # Number of years in a century
optimized_sigma = result.x[0] # The optimized value for sigma
optimized_eccentricity = result.x[1]*e

# Calculate the number of orbits Mercury completes in a century
orbits_per_century = (days_per_year / orbital_period_days) * years_per_century

# Calculate the precession per orbit in radians
precession_per_orbit_radians = 2 * np.pi * (optimized_sigma - 1)

# Convert precession per orbit to arcseconds
precession_per_orbit_arcseconds = np.degrees(precession_per_orbit_radians) * 3600

# Total precession in arcseconds per century
total_precession_arcseconds_per_century = precession_per_orbit_arcseconds * orbits_per_century
total_precession_arcseconds_per_century 


Optimized sigma: [1.00000009 0.99999979]


46.32506671506538

# FRAME DRAGGING CONTRIBUTION

In [5]:
import numpy as np

def lense_thirring_precession(G, J, c, a, e):
    """
    Calculate the Lense-Thirring precession for Mercury due to the Sun's rotation.

    Parameters:
    G (float): Gravitational constant (m^3 kg^-1 s^-2)
    J (float): Angular momentum of the Sun (kg m^2 s^-1)
    c (float): Speed of light (m/s)
    a (float): Semi-major axis of Mercury's orbit (m)
    e (float): Eccentricity of Mercury's orbit

    Returns:
    float: Lense-Thirring precession in arcseconds per century
    """
    # Calculate the Lense-Thirring precession in radians per revolution
    delta_phi_LT_rad = (2 * G * J) / (c**2 * a**3 * (1 - e**2)**(3/2))
    
    # Convert radians per revolution to arcseconds per revolution
    delta_phi_LT_arcsec = delta_phi_LT_rad * (360 * 3600) / (2 * np.pi)
    
    # Convert arcseconds per revolution to arcseconds per century
    # Mercury completes about 415 revolutions per century
    revolutions_per_century = 415
    delta_phi_LT_arcsec_per_century = delta_phi_LT_arcsec * revolutions_per_century
    
    return delta_phi_LT_arcsec_per_century

# Constants
G = 6.67430e-11  # gravitational constant in m^3 kg^-1 s^-2
c = 299792458    # speed of light in m/s
a = 57.91e9      # semi-major axis of Mercury's orbit in meters (57.91 million km)
e = 0.205630     # eccentricity of Mercury's orbit

# Angular momentum of the Sun, approximated as J = I * omega
# I for the Sun (moment of inertia) ~ 0.070 (I/MR^2) * M * R^2
# M = 1.989e30 kg (mass of the Sun), R = 696.34e6 m (radius of the Sun)
# omega = 2.865e-6 rad/s (angular velocity of the Sun, 25-day period at the equator)
M_sun = 1.989e30*1E9  # mass of the Sun in kg
R_sun = 696.34e6  # radius of the Sun in meters
omega_sun = 2.865e-6  # angular velocity of the Sun in rad/s
I_sun = 0.070 * M_sun * R_sun**2  # moment of inertia of the Sun

J_sun = I_sun * omega_sun  # angular momentum of the Sun

# Calculate Lense-Thirring Precession
delta_phi_LT = lense_thirring_precession(G, J_sun, c, a, e)
print(f"Lense-Thirring precession of Mercury's perihelion due to the Sun's rotation: {delta_phi_LT:.12f} arcseconds per century")


Lense-Thirring precession of Mercury's perihelion due to the Sun's rotation: 0.135098853567 arcseconds per century


# Black Hole

In [6]:
import numpy as np

# Constants
G = 6.67430e-11  # gravitational constant, m^3 kg^-1 s^-2
c = 299792458    # speed of light, m/s
a = 57.91e9      # semi-major axis of Mercury's orbit, m
e = 0.205630     # eccentricity of Mercury's orbit

# Black hole parameters
M_bh = 1e9 * 1.989e30  # mass of the black hole, kg (1 billion solar masses)
R_s = 2 * G * M_bh / c**2  # Schwarzschild radius of the black hole, m

# Estimating the black hole's angular velocity
# Assuming it rotates once per its own 'day', which we'll assume is 1000 seconds for calculation
# This is a purely hypothetical and illustrative assumption
T_bh = 1000  # rotational period of the black hole, seconds
omega_bh = 2 * np.pi / T_bh  # angular velocity, rad/s

# Black hole's angular momentum (J = I * omega)
# Using the simple formula for angular momentum of a rotating sphere, J = 2/5 M R^2 omega
# This is not precise for black holes but gives us a ballpark figure for illustrative purposes
I_bh = (2/5) * M_bh * R_s**2  # moment of inertia of the black hole
J_bh = I_bh * omega_bh  # angular momentum of the black hole

# Lense-Thirring Precession calculation
delta_phi_LT_rad = (2 * G * J_bh) / (c**2 * a**3 * (1 - e**2)**(1.5))
delta_phi_LT_arcsec = delta_phi_LT_rad * (360 * 3600) / (2 * np.pi)  # convert radians to arcseconds
revolutions_per_century = 415  # Mercury completes approximately 415 revolutions per century
delta_phi_LT_arcsec_per_century = delta_phi_LT_arcsec * revolutions_per_century  # total precession per century

print(f"Lense-Thirring precession at Mercury's orbit due to a 1 billion solar mass black hole: {delta_phi_LT_arcsec_per_century:.6f} arcseconds per century")


Lense-Thirring precession at Mercury's orbit due to a 1 billion solar mass black hole: 30470835591.638950 arcseconds per century
